In [ ]:
library(magrittr)

library(corrplot)

library(preprocessCore)

#library(DNAcopy)

library(RColorBrewer)

library(reshape)

library(ggplot2)

In [ ]:
bin <- "10kb"

In [ ]:
binsize <- 10000

In [ ]:
genome <- "mm10"

In [ ]:
genome_file <-  "/data/zqwen/genome/mm10/mm10.chrom.filter.sort.sizes" 

In [ ]:
nri.dir <- '/data/zqwen/subnucleosome/repli1/06.nri/'

In [ ]:
plot.dir <- '/data/zqwen/subnucleosome/repli1/07.plot/'

In [ ]:
fn <- "trp-60-pd" ## prefix of file

In [ ]:
countfile <- paste0(nri.dir, fn, ".", genome, ".", bin, ".raw.counts.txt")

In [ ]:
df <- read.delim(countfile, header = TRUE)

In [ ]:
names(df)<- c("Chr", "Start", "End", "g140", "g250")

In [ ]:
bincounts <- rowSums(df[,4:ncol(df)])

In [ ]:
index <- (bincounts > 0)

In [ ]:
df <- df[index,]

### Calculate nrs for each bin

In [ ]:
df <- within(df, nrs <- (g250 - g140)/(g250 + g140))

In [ ]:
df$nri <- scale(df$nrs) %>% as.vector()

### Register the smoothed nrs and nri into bedGraph files

In [ ]:
for (value in c("nrs","nri")){
    
    message("Writing raw: ", value)
    
    dfsub <- df[,c("Chr" , "Start" , "End" , value)]
    
    filename <- paste0(nri.dir,  fn, ".", genome, ".", bin, ".", value,".bdg") 
        
    write.table(dfsub, filename, sep= "\t", row.names=FALSE, quote=FALSE, col.names = FALSE)
    
    message("Loess smoothing: ", value)
    
    chrs <- unique(dfsub$Chr)
    
    binnum <- 10
    
    Loess=data.frame()

    for(Chr in chrs){
        message("Current chrom: " , Chr)
        subnrs <- dfsub[dfsub$Chr == Chr,]
        lspan=binnum*binsize/(max(subnrs$Start)-min(subnrs$Start)) 
        nrsfit=loess(subnrs[,4] ~ subnrs$Start, span=lspan, surface = c("direct"))
        nrilsm <- data.frame(c(rep(Chr,times=nrsfit$n)), nrsfit$x, subnrs[which(subnrs$Start %in% nrsfit$x), 3], nrsfit$fitted)
        if(nrow(Loess)!=0){ Loess = rbind(Loess,nrilsm)}
        if(nrow(Loess)==0){ Loess = nrilsm}
        }

    names(Loess)=c("Chr" , "Start" , "End" , "Fitted")
    
    bdgname <- paste0(nri.dir,  fn, ".", genome, ".", bin, ".", value, ".", binnum, "bins.smoothed.bdg")
    
    write.table(Loess, bdgname, sep= "\t", row.names=FALSE, quote=FALSE, col.names = FALSE)
    
    bwname <- paste0(nri.dir,  fn, ".", genome, ".", bin, ".", value, ".", binnum, "bins.smoothed.bw")
    
    system2("bedGraphToBigWig", paste(bdgname, genome_file, bwname, sep=" "), wait = TRUE, stderr = TRUE, stdout = TRUE)
}